In [90]:
import pandas as pd
import numpy as np

In [91]:
def preprocess(df, col):

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    # df['ts'] = df['timestamp'].astype(int)

    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day_of_week'] = df.timestamp.dt.weekday
    df['week_of_year'] = df.timestamp.dt.weekofyear
    df['timestamp'] = df['timestamp'].astype(np.int64)
    df = df.sort_values(by=col).reset_index(drop=True)
    df = df.fillna(method='bfill')
    df = df.fillna(method='ffill')
    df["ratio_life_dash_full_sq"] = np.divide(df["life_sq"], df["full_sq"], out=np.zeros_like(df["life_sq"]), where=df["full_sq"]!=0)
    df["ration_kitchen_dash_full_sq"] =  np.divide(df["kitch_sq"], df["full_sq"], out=np.zeros_like(df["life_sq"]), where=df["full_sq"]!=0)

    df['age'] = df['year'] - df["build_year"]
    df['some_extra_sqr'] = df["full_sq"] - df["life_sq"]
    df = df.fillna(method='bfill')
    df = df.fillna(method='ffill')
    # df = df.fillna(df.mean(), inplace=True)
    return df

In [92]:
train1 = pd.read_csv('HW_train_main_data.csv')
train2 = pd.read_csv('HW_train_additional_data.csv')

In [93]:
train = train1.merge(train2, how='left', on='id')

In [94]:
test = pd.read_csv('HW_test_main_data.csv')
tes2 = pd.read_csv('HW_test_additional_data.csv')

In [95]:
test = test.merge(tes2, how='left', on='id')

In [96]:
test = preprocess(test, 'life_sq')

C:\Users\denko\AppData\Local\Temp/ipykernel_3924/3061532774.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = df.timestamp.dt.weekofyear
C:\Users\denko\AppData\Local\Temp/ipykernel_3924/3061532774.py:10: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  df['timestamp'] = df['timestamp'].astype(np.int64)


In [97]:
train = preprocess(train, 'price')

C:\Users\denko\AppData\Local\Temp/ipykernel_3924/3061532774.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = df.timestamp.dt.weekofyear
C:\Users\denko\AppData\Local\Temp/ipykernel_3924/3061532774.py:10: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  df['timestamp'] = df['timestamp'].astype(np.int64)


In [98]:
from sklearn.model_selection import train_test_split

In [99]:
cols = train.columns[(train.columns != 'price') & (train.columns != 'year_month')]
cols

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq', 'apartment condition',
       'sub_area', 'population', 'indust_part', 'preschool_facilities',
       'school_facilities', 'hospital_beds_raion', 'healthcare_facilities',
       'university_num', 'sport_objects_facilities',
       'additional_education_facilities', 'culture_objects_facilities',
       'shopping_centers_facilities', 'office_num', 'green_part', 'prom_part',
       'cafe_count', 'church_facilities', 'mosque', 'leisure_facilities',
       'year', 'month', 'day_of_week', 'week_of_year',
       'ratio_life_dash_full_sq', 'ration_kitchen_dash_full_sq', 'age',
       'some_extra_sqr'],
      dtype='object')

In [101]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 4,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'min_child_weight':1,
    'silent': 1,
    'seed':0
}

In [102]:
import xgboost as xgb

In [108]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.fit_transform(test)
# train['timestamp'] = scaler.fit_transform(train['timestamp'])

In [110]:
train

array([[ 0.47515272,  0.52272112,  0.43229455, ...,  0.05268582,
         0.00743284,  0.3702824 ],
       [-1.44186363, -2.01033544, -0.44443179, ..., -0.13559118,
         0.00737278, -0.17307125],
       [-0.48899408, -2.0073728 , -0.21235717, ..., -0.14235113,
         0.00737278, -0.14590357],
       ...,
       [ 1.27656848,  1.31078316,  4.30020489, ..., -0.06816766,
         0.00791331,  1.53849274],
       [-1.56953129, -0.68899833,  3.42347855, ..., -0.04958965,
         0.0079013 ,  2.73387077],
       [ 1.16006259, -1.78813749,  0.04550352, ...,  0.22919552,
         0.0078953 ,  0.12577325]])

In [109]:
X_train, X_test, y_train, y_test = train_test_split(
    train[cols],
    train.price, test_size=1425, random_state=42)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [105]:
xgb_train = xgb.DMatrix(X_train, y_train, feature_names = cols, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test, y_test, feature_names = cols, enable_categorical=True)

In [106]:
evallist = [(xgb_test, 'eval'), (xgb_train, 'train')]

In [107]:
model_2 = xgb.train(params = xgb_params,
                    dtrain = xgb_train,
                    num_boost_round = 300,
                    evals = evallist,
                    early_stopping_rounds = 10,
                    verbose_eval = 10)

[15:39:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:39:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-rmse:7885143.50000	train-rmse:8679628.00000
[10]	eval-rmse:5203258.00000	train-rmse:5882132.00000
[20]	eval-rmse:3801437.00000	train-rmse:4378543.00000
[30]	eval-rmse:3153103.75000	train-rmse:3615511.75000
[40]	eval-rmse:2870713.75000	train-rmse:3224390.50000
[50]	eval-rmse:2729835.75000	train-rmse:3015962.75000
[60]	eval-rmse:2673673.75000	train-rmse:2887694.50000
[70]	eval-rmse:2637913.00000	train-rmse:2805288.00000
[80]	e